<a href="https://colab.research.google.com/github/31indianaSKKU/NBA_CareerLength/blob/main/NBA_Draft_Combine_Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 이 부분은 처음 한번만 실행하면 됌.
# "The reason is that the last Ubuntu update update supports chromium driver just via snap."
# 최근 우분투 업데이트에서 크롬 드라이버 설치를 snap을 이용해서만 하도록 바뀜
# 고로 snap 없이 설치하는 아래 우회 코드로 변경
# 출처 : https://colab.research.google.com/drive/1cbEvuZOhkouYLda3RqiwtbM-o9hxGLyC
# 출처2 : https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver


# Install selenium
pip install selenium
pip install webdriver-manager

In [ ]:
#시즌별 반복 시도_코드 반영
#파일생성 시 고민: 1안. 모든 시즌을 하나의 파일에.  2안. 시즌별로 파일생성. _2안 선택
#이 코드는 에러없이 잘 가져오지만 파일이 마지막 시즌만 저장되고 있음.->해결
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
import re
from functools import reduce
import time

class Strength_Agility_results:
    def __init__(self, Season, Name, Pos, LANE_AGILITY_sec, SHUTTLE_RUN_sec, THREE_QUATER_SPRINT, STANDING_VERTICAL_LEAP_inch, MAX_VERTICAL_LEAP_inch, MAX_BENCH_PRESS):
        self.Season = Season
        self.Name = Name
        self.Pos = Pos
        self.LANE_AGILITY_sec = LANE_AGILITY_sec
        self.SHUTTLE_RUN_sec = SHUTTLE_RUN_sec
        self.THREE_QUATER_SPRINT = THREE_QUATER_SPRINT
        self.STANDING_VERTICAL_LEAP_inch = STANDING_VERTICAL_LEAP_inch
        self.MAX_VERTICAL_LEAP_inch = MAX_VERTICAL_LEAP_inch
        self.MAX_BENCH_PRESS = MAX_BENCH_PRESS

    def to_list(self):
        return [self.Season, self.Name, self.Pos, self.LANE_AGILITY_sec, self.SHUTTLE_RUN_sec, self.THREE_QUATER_SPRINT, self.STANDING_VERTICAL_LEAP_inch, self.MAX_VERTICAL_LEAP_inch, self.MAX_BENCH_PRESS]

class Anthro_results:   
    def __init__(self, Season, Name, Pos, Body_fat_pct, Hand_Length_inch, Hand_width_inch, Height_wo_Shoes, Height_w_Shoes, Standing_Reach_inch, Weight_lbs, Wingspan_inch):
        self.Season = Season
        self.Name = Name
        self.Pos = Pos
        self.Body_fat_pct = Body_fat_pct
        self.Hand_Length_inch = Hand_Length_inch
        self.Hand_width_inch = Hand_width_inch
        self.Height_wo_Shoes = Height_wo_Shoes
        self.Height_w_Shoes = Height_w_Shoes
        self.Standing_Reach_inch = Standing_Reach_inch
        self.Weight_lbs = Weight_lbs
        self.Wingspan_inch = Wingspan_inch

    def to_list(self):
        return [self.Season, self.Name, self.Pos, self.Body_fat_pct, self.Hand_Length_inch, self.Hand_width_inch, self.Height_wo_Shoes, self.Height_w_Shoes, self.Standing_Reach_inch, self.Weight_lbs, self.Wingspan_inch]

class Non_Stationary_results:   
    def __init__(self, Season, Name, Pos, OFF_DRIBBLE_COELLEGE_BREAK_left_pct, OFF_DRIBBLE_COELLEGE_BREAK_right_pct, OFF_DRIBBLE_COELLEGE_TOP_KEY_pct, OFF_DRIBBLE_FIFTEEN_BREAK_left_pct, 
                 OFF_DRIBBLE_FIFTEEN_BREAK_right_pct, OFF_DRIBBLE_FIFTEEN_TOP_KEY_pct, ON_THE_MOVE_COLLEGE_pct, ON_THE_MOVE_FIFTEEN_pct):
        self.Season = Season
        self.Name = Name
        self.Pos = Pos
        self.OFF_DRIBBLE_COELLEGE_BREAK_left_pct = OFF_DRIBBLE_COELLEGE_BREAK_left_pct
        self.OFF_DRIBBLE_COELLEGE_BREAK_right_pct = OFF_DRIBBLE_COELLEGE_BREAK_right_pct
        self.OFF_DRIBBLE_COELLEGE_TOP_KEY_pct = OFF_DRIBBLE_COELLEGE_TOP_KEY_pct
        self.OFF_DRIBBLE_FIFTEEN_BREAK_left_pct = OFF_DRIBBLE_FIFTEEN_BREAK_left_pct
        self.OFF_DRIBBLE_FIFTEEN_BREAK_right_pct = OFF_DRIBBLE_FIFTEEN_BREAK_right_pct
        self.OFF_DRIBBLE_FIFTEEN_TOP_KEY_pct = OFF_DRIBBLE_FIFTEEN_TOP_KEY_pct
        self.ON_THE_MOVE_COLLEGE_pct = ON_THE_MOVE_COLLEGE_pct
        self.ON_THE_MOVE_FIFTEEN_pct = ON_THE_MOVE_FIFTEEN_pct

    def to_list(self):
        return [self.Season, self.Name, self.Pos, self.OFF_DRIBBLE_COELLEGE_BREAK_left_pct, self.OFF_DRIBBLE_COELLEGE_BREAK_right_pct, self.OFF_DRIBBLE_COELLEGE_TOP_KEY_pct,
                self.OFF_DRIBBLE_FIFTEEN_BREAK_left_pct, self.OFF_DRIBBLE_FIFTEEN_BREAK_right_pct, self.OFF_DRIBBLE_FIFTEEN_TOP_KEY_pct,
               self.ON_THE_MOVE_COLLEGE_pct, self.ON_THE_MOVE_FIFTEEN_pct]

class Spot_Up_results:   
    def __init__(self, Season, Name, Pos, NBA_BREAK_left_pct, NBA_BREAK_right_pct, NBA_CORNER_left_pct, NBA_CORNER_right_pct, NBA_TOP_KEY_pct,
                 COLLEGE_BREAK_left_pct, COLLEGE_BREAK_right_pct, COLLEGE_CORNER_left_pct, COLLEGE_CORNER_right_pct, COLLEGE_TOP_KEY_pct,
                 FIFTEEN_BREAK_left_pct, FIFTEEN_BREAK_right_pct, FIFTEEN_CORNER_left_pct, FIFTEEN_CORNER_right_pct, FIFTEEN_TOP_KEY_pct):
        self.Season = Season
        self.Name = Name
        self.Pos = Pos
        self.NBA_BREAK_left_pct = NBA_BREAK_left_pct
        self.NBA_BREAK_right_pct = NBA_BREAK_right_pct
        self.NBA_CORNER_left_pct = NBA_CORNER_left_pct
        self.NBA_CORNER_right_pct = NBA_CORNER_right_pct
        self.NBA_TOP_KEY_pct = NBA_TOP_KEY_pct
        self.COLLEGE_BREAK_left_pct = COLLEGE_BREAK_left_pct
        self.COLLEGE_BREAK_right_pct = COLLEGE_BREAK_right_pct
        self.COLLEGE_CORNER_left_pct = COLLEGE_CORNER_left_pct
        self.COLLEGE_CORNER_right_pct = COLLEGE_CORNER_right_pct
        self.COLLEGE_TOP_KEY_pct = COLLEGE_TOP_KEY_pct
        self.FIFTEEN_BREAK_left_pct = FIFTEEN_BREAK_left_pct
        self.FIFTEEN_BREAK_right_pct = FIFTEEN_BREAK_right_pct
        self.FIFTEEN_CORNER_left_pct = FIFTEEN_CORNER_left_pct
        self.FIFTEEN_CORNER_right_pct = FIFTEEN_CORNER_right_pct
        self.FIFTEEN_TOP_KEY_pct = FIFTEEN_TOP_KEY_pct

    def to_list(self):
        return [self.Season, self.Name, self.Pos, self.NBA_BREAK_left_pct, self.NBA_BREAK_right_pct, self.NBA_CORNER_left_pct, self.NBA_CORNER_right_pct, 
                self.NBA_TOP_KEY_pct, self.COLLEGE_BREAK_left_pct, self.COLLEGE_BREAK_right_pct, self.COLLEGE_CORNER_left_pct, self.COLLEGE_CORNER_right_pct,
                self.COLLEGE_TOP_KEY_pct, self.FIFTEEN_BREAK_left_pct, self.FIFTEEN_BREAK_right_pct, self.FIFTEEN_CORNER_left_pct, self.FIFTEEN_CORNER_right_pct,
                self.FIFTEEN_TOP_KEY_pct]

# Chrome 웹드라이버 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('window-size=1080,3840')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('disable-dev-shm-usage')
chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
browser = webdriver.Chrome('chromedriver', options=chrome_options)

# 데이터 수집을 위한 크롤링 작업 수행
strength_agility_list = []
anthro_list = []
non_stationary_shooting_list = []
spot_up_shooting_list = []

for year in range(2021, 2023):
    season = f"{year}-{str(year+1)[-2:]}"

    # 1. Strength_Agility 작업 수행
    # browser.get("https://www.nba.com/stats/draft/combine-strength-agility")
    browser.get(f"https://www.nba.com/stats/draft/combine-strength-agility?SeasonYear={season}")
    browser.implicitly_wait(3)
    Accept_button = browser.find_elements(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
    table = browser.find_element(By.XPATH, '//*[@id="__next"]') #//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[3]/table
    trs = table.find_elements(By.XPATH, './/tr')

    # strength_agility_list = []
    #데이터에 추가할 Season값 가져오기
    url = browser.current_url
    pattern = r"SeasonYear=(\d{4}-\d{2})"

    match = re.search(pattern, url)
    if match:
        Season = match.group(1)

    while True:
        try:
            browser.implicitly_wait(5)
            tbody = browser.find_element(By.CLASS_NAME, 'Crom_body__UYOcU')
            trs = tbody.find_elements(By.TAG_NAME, 'tr')

            for tr in trs:
                tds = tr.find_elements(By.TAG_NAME, 'td')
                if len(tds) == 0:
                    continue
                else:
                    Name = tds[0].text
                    Pos = tds[1].text
                    LANE_AGILITY_sec = tds[2].text
                    SHUTTLE_RUN_sec = tds[3].text
                    THREE_QUATER_SPRINT = tds[4].text
                    STANDING_VERTICAL_LEAP_inch = tds[5].text
                    MAX_VERTICAL_LEAP_inch = tds[6].text
                    MAX_BENCH_PRESS = tds[7].text

                    time.sleep(1) 

                    strength_agility = Strength_Agility_results(Season, Name, Pos, LANE_AGILITY_sec, SHUTTLE_RUN_sec, THREE_QUATER_SPRINT, STANDING_VERTICAL_LEAP_inch, MAX_VERTICAL_LEAP_inch, MAX_BENCH_PRESS)
                    strength_agility_list.append(strength_agility.to_list())

                    print(strength_agility.to_list())

            break

        except Exception as e:
            print(e)

for year in range(2021, 2023):
    season = f"{year}-{str(year+1)[-2:]}"

    # 2. Anthro 작업 수행
    browser.get(f"https://www.nba.com/stats/draft/combine-anthro?SeasonYear={season}")
    browser.implicitly_wait(3)
    Accept_button = browser.find_elements(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
    table = browser.find_element(By.XPATH, '//*[@id="__next"]') #//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[3]/table
    trs = table.find_elements(By.XPATH, './/tr')
    # print(table)

    # anthro_list = []
    url = browser.current_url
    pattern = r"SeasonYear=(\d{4}-\d{2})"

    match = re.search(pattern, url)
    if match:
        Season = match.group(1)


    while True:
        try:
            browser.implicitly_wait(5)
            tbody = browser.find_element(By.CLASS_NAME, 'Crom_body__UYOcU')
            trs = tbody.find_elements(By.TAG_NAME, 'tr')

            for tr in trs:
                tds = tr.find_elements(By.TAG_NAME, 'td')
                if len(tds) == 0:
                    continue
                else:
                    Name = tds[0].text
                    Pos = tds[1].text
                    Body_fat_pct = tds[2].text
                    Hand_Length_inch = tds[3].text
                    Hand_width_inch = tds[4].text
                    Height_wo_Shoes = tds[5].text
                    Height_w_Shoes = tds[6].text
                    Standing_Reach_inch = tds[7].text
                    Weight_lbs = tds[8].text
                    Wingspan_inch = tds[9].text

                    time.sleep(1) 

                    anthro = Anthro_results(Season, Name, Pos, Body_fat_pct, Hand_Length_inch, Hand_width_inch, Height_wo_Shoes, Height_w_Shoes, Standing_Reach_inch, Weight_lbs, Wingspan_inch)
                    anthro_list.append(anthro.to_list())

                    print(anthro.to_list())
            break

        except Exception as e:
            print(e)        

for year in range(2021, 2023):
    season = f"{year}-{str(year+1)[-2:]}"

    # 3. Non_stationary_shooting 작업 수행
    browser.get(f"https://www.nba.com/stats/draft/combine-non-stationary?SeasonYear={season}")
    browser.implicitly_wait(3)
    Accept_button = browser.find_elements(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
    table = browser.find_element(By.XPATH, '//*[@id="__next"]') #//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[3]/table
    trs = table.find_elements(By.XPATH, './/tr')

    # print(table)
    # non_stationary_shooting_list = []

    url = browser.current_url
    pattern = r"SeasonYear=(\d{4}-\d{2})"

    match = re.search(pattern, url)
    if match:
        Season = match.group(1)


    while True:
        try:
            browser.implicitly_wait(5)
            tbody = browser.find_element(By.CLASS_NAME, 'Crom_body__UYOcU')
            trs = tbody.find_elements(By.TAG_NAME, 'tr')

            for tr in trs:
                tds = tr.find_elements(By.TAG_NAME, 'td')
                if len(tds) == 0:
                    continue
                else:
                    Name = tds[0].text
                    Pos = tds[1].text
                    OFF_DRIBBLE_COELLEGE_BREAK_left_pct = tds[2].text
                    OFF_DRIBBLE_COELLEGE_BREAK_right_pct = tds[3].text
                    OFF_DRIBBLE_COELLEGE_TOP_KEY_pct = tds[4].text
                    OFF_DRIBBLE_FIFTEEN_BREAK_left_pct = tds[5].text
                    OFF_DRIBBLE_FIFTEEN_BREAK_right_pct = tds[6].text
                    OFF_DRIBBLE_FIFTEEN_TOP_KEY_pct = tds[7].text
                    ON_THE_MOVE_COLLEGE_pct = tds[8].text
                    ON_THE_MOVE_FIFTEEN_pct = tds[9].text

                    time.sleep(1) 

                    non_stationary_shooting = Non_Stationary_results(Season, Name, Pos, OFF_DRIBBLE_COELLEGE_BREAK_left_pct, OFF_DRIBBLE_COELLEGE_BREAK_right_pct, 
                                                                    OFF_DRIBBLE_COELLEGE_TOP_KEY_pct, OFF_DRIBBLE_FIFTEEN_BREAK_left_pct, OFF_DRIBBLE_FIFTEEN_BREAK_right_pct,
                                                                    OFF_DRIBBLE_FIFTEEN_TOP_KEY_pct, ON_THE_MOVE_COLLEGE_pct, ON_THE_MOVE_FIFTEEN_pct)
                    non_stationary_shooting_list.append(non_stationary_shooting.to_list())

                    print(non_stationary_shooting.to_list())
            break
              
        except Exception as e:
            print(e)        

for year in range(2000, 2003):  #from-to
    season = f"{year}-{str(year+1)[-2:]}"

    # 4. Spot_Up_shooting 작업 수행
    browser.get(f"https://www.nba.com/stats/draft/combine-spot-up?SeasonYear={season}")
    browser.implicitly_wait(3)
    Accept_button = browser.find_elements(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
    table = browser.find_element(By.XPATH, '//*[@id="__next"]') #//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[3]/table
    trs = table.find_elements(By.XPATH, './/tr')

    # print(table)
    # spot_up_shooting_list = []

    url = browser.current_url
    pattern = r"SeasonYear=(\d{4}-\d{2})"

    match = re.search(pattern, url)
    if match:
        Season = match.group(1)

    while True:
        try:
            browser.implicitly_wait(5)
            tbody = browser.find_element(By.CLASS_NAME, 'Crom_body__UYOcU')
            trs = tbody.find_elements(By.TAG_NAME, 'tr')

            for tr in trs:
                tds = tr.find_elements(By.TAG_NAME, 'td')
                if len(tds) == 0:
                    continue
                else:
                    Name = tds[0].text
                    Pos = tds[1].text
                    NBA_BREAK_left_pct = tds[2].text
                    NBA_BREAK_right_pct = tds[3].text
                    NBA_CORNER_left_pct = tds[4].text
                    NBA_CORNER_right_pct = tds[5].text
                    NBA_TOP_KEY_pct = tds[6].text
                    COLLEGE_BREAK_left_pct = tds[7].text
                    COLLEGE_BREAK_right_pct = tds[8].text
                    COLLEGE_CORNER_left_pct = tds[9].text
                    COLLEGE_CORNER_right_pct = tds[10].text
                    COLLEGE_TOP_KEY_pct = tds[11].text
                    FIFTEEN_BREAK_left_pct = tds[12].text
                    FIFTEEN_BREAK_right_pct = tds[13].text
                    FIFTEEN_CORNER_left_pct = tds[14].text
                    FIFTEEN_CORNER_right_pct = tds[15].text
                    FIFTEEN_TOP_KEY_pct = tds[16].text

                    time.sleep(1) 
                        
                    spot_up_shooting = Spot_Up_results(Season, Name, Pos, NBA_BREAK_left_pct, NBA_BREAK_right_pct, NBA_CORNER_left_pct, NBA_CORNER_right_pct, NBA_TOP_KEY_pct,
                                                                COLLEGE_BREAK_left_pct, COLLEGE_BREAK_right_pct, COLLEGE_CORNER_left_pct, COLLEGE_CORNER_right_pct, COLLEGE_TOP_KEY_pct,
                                                                FIFTEEN_BREAK_left_pct, FIFTEEN_BREAK_right_pct, FIFTEEN_CORNER_left_pct, FIFTEEN_CORNER_right_pct,FIFTEEN_TOP_KEY_pct)
                    spot_up_shooting_list.append(spot_up_shooting.to_list())

                    print(spot_up_shooting.to_list())
            break
        except Exception as e:
            print(e)   

    # 데이터프레임 생성
    strength_agility_df = pd.DataFrame(strength_agility_list, columns=['Season', 'Name', 'Pos', 'LANE_AGILITY_sec', 'SHUTTLE_RUN_sec', 'THREE_QUATER_SPRINT',
                                                                        'STANDING_VERTICAL_LEAP_inch', 'MAX_VERTICAL_LEAP_inch', 'MAX_BENCH_PRESS'])
    anthro_df = pd.DataFrame(anthro_list, columns=['Season', 'Name', 'Pos', 'Body_fat_pct', 'Hand_Length_inch', 'Hand_width_inch', 'Height_wo_Shoes', 'Height_w_Shoes',
                                                  'Standing_Reach_inch', 'Weight_lbs', 'Wingspan_inch'])
    non_stationary_df = pd.DataFrame(non_stationary_shooting_list, columns=['Season', 'Name', 'Pos', 'OFF_DRIBBLE_COELLEGE_BREAK_left_pct', 'OFF_DRIBBLE_COELLEGE_BREAK_right_pct',
                                                                            'OFF_DRIBBLE_COELLEGE_TOP_KEY_pct', 'OFF_DRIBBLE_FIFTEEN_BREAK_left_pct',
                                                                            'OFF_DRIBBLE_FIFTEEN_BREAK_right_pct', 'OFF_DRIBBLE_FIFTEEN_TOP_KEY_pct',
                                                                              'ON_THE_MOVE_COLLEGE_pct', 'ON_THE_MOVE_FIFTEEN_pct'])
    spot_up_df = pd.DataFrame(spot_up_shooting_list, columns=['Season', 'Name', 'Pos', 'NBA_BREAK_left_pct', 'NBA_BREAK_right_pct', 'NBA_CORNER_left_pct',
                                                              'NBA_CORNER_right_pct', 'NBA_TOP_KEY_pct', 'COLLEGE_BREAK_left_pct', 'COLLEGE_BREAK_right_pct',
                                                              'COLLEGE_CORNER_left_pct', 'COLLEGE_CORNER_right_pct', 'COLLEGE_TOP_KEY_pct',
                                                              'FIFTEEN_BREAK_left_pct', 'FIFTEEN_BREAK_right_pct', 'FIFTEEN_CORNER_left_pct',
                                                              'FIFTEEN_CORNER_right_pct', 'FIFTEEN_TOP_KEY_pct'])

    # 데이터프레임을 선수 이름과 Season을 기준으로 열 방향으로 병합
    dfs = [strength_agility_df, anthro_df, non_stationary_df, spot_up_df]
    merged_df = reduce(lambda left, right: pd.merge(left, right, on=['Name', 'Season', 'Pos'], how='outer'), dfs)

    # 파일로 저장
    merged_df.to_csv('Draft_Combine_Results.csv', index=False)



